#**Notebook for the first assignment_Third Part**

In [0]:

#installing  wikipedia module 
!pip install wikipedia

In [0]:
import pandas as pd 
import numpy as np
import wikipedia as wp
from bs4 import BeautifulSoup
import requests
import io



we scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

# **Entering the h1 element in wp.page(h1)** 

In [0]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# **Determine the index of the table**

In [0]:
df = pd.read_html(html, header = 0)[0]

# **Processing the cells that have only an assigned borough**

In [0]:
df = df[df.Borough != 'Not assigned']

#**Duplicate Postcode**

In [0]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()


#**Setting neighborhood = borough if a cell has a borough but a Not assigned neighborhood**

In [25]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [26]:
df.shape

(103, 3)

#**Get geographical coordinates for each postcode**

We need first to get the latitude and the longitude coordinates of each neighborhood

In [0]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

#**Rename the first column to allow merging dataframes on Postcode**

In [0]:
c.columns = ['Postcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postcode')


# **Reorder column names and show the dataframe**

In [29]:
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [30]:
df.shape

(103, 5)


Explore and cluster the neighborhood

In [87]:
df=df.loc[df['Borough'].str.contains('Toronto')]
df.head()
Out[87]:
Postcode	Borough	Neighbourhood	Latitude	Longitude
37	M4E	East Toronto	East Toronto	43.676357	-79.293031
41	M4K	East Toronto	East Toronto,East Toronto	43.679557	-79.352188
42	M4L	East Toronto	East Toronto,East Toronto	43.668999	-79.315572
43	M4M	East Toronto	East Toronto	43.659526	-79.340923
44	M4N	Central Toronto	Central Toronto	43.728020	-79.388790
Cluster the neighborhood
In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
Group data by neighborhood
In [110]:
toronto_grouped =df.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()
Out[110]:
Neighbourhood	Latitude	Longitude
0	Central Toronto	43.714435	-79.398078
1	Central Toronto,Central Toronto	43.693261	-79.397234
2	Central Toronto,Central Toronto,Central Toronto	43.672710	-79.405678
3	Central Toronto,Central Toronto,Central Toront...	43.686412	-79.400049
4	Downtown Toronto	43.659131	-79.382704
In [111]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# **filtering data**


In [32]:
df=df.loc[df['Borough'].str.contains('Toronto')]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# **Cluster the neighborhood**

In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# **Group data by neighborhood**

In [34]:
toronto_grouped =df.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Latitude,Longitude
0,"Adelaide, King, Richmond",43.650571,-79.384568
1,Berczy Park,43.644771,-79.373306
2,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 1, 2, 2, 2, 0, 0, 2], dtype=int32)

# **Create Map of Cluster**

In [36]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped

,Cluster Labels,Neighbourhood,Latitude,Longitude
0,2,"Adelaide, King, Richmond",43.650571,-79.384568
1,2,Berczy Park,43.644771,-79.373306
2,0,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,1,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
4,2,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
5,2,"Cabbagetown, St. James Town",43.667967,-79.367675
6,2,Central Bay Street,43.657952,-79.387383
7,0,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
8,0,Christie,43.669542,-79.422564
9,2,Church and Wellesley,43.665860,-79.383160


# **Create map**

In [39]:
map_clusters = folium.Map(location=[toronto_grouped['Latitude'].mean(),toronto_grouped['Longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['Latitude'],toronto_grouped['Longitude'],toronto_grouped['Neighbourhood'],toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
toronto_merged =df

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged =toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.head() # check the last columns!